In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime  
import pickle
from math import floor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sqlalchemy import text

Inserting data base information and connecting to it

In [2]:

URI = "dbbikes2.cytgvbje9wgu.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes2"
USER = "admin"
PASSWORD = "DublinBikes1"

engine = create_engine("mysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

Want to get features to compare in train_test split, I will use datetime to try predict availability first

In [3]:
# Creating get_features function
def get_features():
    # Get datetime and station number for data
    features_raw = engine.execute("SELECT datetime, number FROM availability2;")

    # Create list to store station numbers, days and hours in.
    numbers_days_hours = []
    for row in features_raw:
        
        # Will store each trio in
        number_day_hour = []
        
        # Split epoch time into day and hour
        epoch_time = row[0]
        day = (floor(epoch_time / 86400) + 4) % 7
        number_day_hour.append(row[1])
        number_day_hour.append(day)
        number_day_hour.append(datetime.datetime.fromtimestamp(row[0]).hour)
        numbers_days_hours.append(number_day_hour)
    return numbers_days_hours

def get_availability():
    availability_raw = engine.execute("SELECT available_bikes FROM availability2;")
    # This returns the number of available bikes
    availability = []
    for row in availability_raw:
        availability.append(row[0])
    return availability
                                     

In [4]:

X = get_features()
y = get_availability()

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

2023-04-14 12:19:19,857 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-14 12:19:19,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 12:19:20,374 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-14 12:19:20,375 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 12:19:20,471 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-14 12:19:20,471 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 12:19:20,804 INFO sqlalchemy.engine.Engine SELECT datetime, number FROM availability2;
2023-04-14 12:19:20,804 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 12:19:29,135 INFO sqlalchemy.engine.Engine SELECT available_bikes FROM availability2;
2023-04-14 12:19:29,135 INFO sqlalchemy.engine.Engine [raw sql] ()


[[82, 1, 20],
 [63, 3, 6],
 [102, 5, 8],
 [57, 5, 16],
 [36, 2, 3],
 [9, 0, 19],
 [22, 4, 15],
 [82, 5, 3],
 [117, 6, 10],
 [45, 1, 7],
 [111, 5, 20],
 [111, 6, 13],
 [109, 4, 14],
 [75, 5, 22],
 [48, 3, 0],
 [55, 3, 23],
 [64, 6, 13],
 [108, 1, 21],
 [92, 4, 15],
 [92, 0, 1],
 [61, 0, 12],
 [101, 4, 5],
 [99, 4, 2],
 [82, 4, 23],
 [107, 1, 12],
 [71, 6, 19],
 [36, 4, 17],
 [96, 6, 17],
 [33, 0, 11],
 [36, 0, 0],
 [20, 3, 5],
 [68, 6, 15],
 [45, 5, 10],
 [95, 6, 7],
 [99, 1, 10],
 [54, 6, 12],
 [16, 2, 22],
 [23, 6, 3],
 [60, 0, 12],
 [114, 5, 5],
 [100, 4, 19],
 [33, 0, 7],
 [97, 5, 5],
 [107, 3, 5],
 [52, 0, 2],
 [116, 2, 18],
 [115, 0, 11],
 [92, 4, 13],
 [24, 3, 12],
 [85, 0, 16],
 [23, 4, 14],
 [5, 6, 14],
 [91, 1, 13],
 [17, 0, 10],
 [64, 4, 22],
 [71, 1, 22],
 [6, 1, 20],
 [79, 0, 21],
 [112, 6, 15],
 [35, 3, 21],
 [67, 0, 0],
 [75, 5, 9],
 [107, 6, 23],
 [69, 2, 22],
 [78, 4, 6],
 [33, 1, 14],
 [88, 5, 8],
 [68, 3, 13],
 [21, 1, 19],
 [3, 4, 2],
 [39, 2, 22],
 [67, 1, 19],
 [98

In [5]:

# fit final model
model = SGDRegressor() # choose your own model here...
model.fit(X_train, y_train)

Xnew = X_test # query features

# make a prediction
ynew = model.predict(X_test)
for i in range(10):
    print("X=%s, Predicted=%s, Actual = %s" % (Xnew[i], ynew[i], y_test[i]))

X=[9, 6, 15], Predicted=14.442703929282217, Actual = 16
X=[75, 1, 17], Predicted=12.316271615231766, Actual = 30
X=[55, 6, 17], Predicted=12.779088651064658, Actual = 5
X=[109, 3, 17], Predicted=10.602300079800731, Actual = 13
X=[69, 2, 13], Predicted=11.666569115053054, Actual = 32
X=[18, 6, 5], Predicted=11.954006377664264, Actual = 13
X=[95, 4, 22], Predicted=12.187903713340646, Actual = 32
X=[12, 1, 6], Predicted=12.874996444114148, Actual = 1
X=[53, 6, 22], Predicted=13.91039585843529, Actual = 10
X=[32, 0, 21], Predicted=15.181588856802032, Actual = 23


In [6]:
# X_train and y_train are the training data
model = SGDRegressor()
model.fit(X_train, y_train)
# Serialize model object into a file called model.pkl on disk using pickle
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# pickle.HIGHEST_PROTOCOL using the highest available protocol
# (we used wb to open file as binary and use a higher pickling protocol)
